# I. Managing indices, mappings, analysis, ranking

## 1. Create index

In [1]:
import elasticsearch

es = elasticsearch.Elasticsearch()

index_name = "codegrepper"
type_name = "answer"

if es.indices.exists(index_name):
    es.indices.delete(index_name)

es.indices.create(index_name)
es.cluster.health(wait_for_status="yellow")

/Users/cpu349/Desktop/PythonProject/venv/lib/python3.9/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.13/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'cluster_name': 'search-javascript',
 'status': 'yellow',
 'timed_out': False,
 'number_of_nodes': 1,
 'number_of_data_nodes': 1,
 'active_primary_shards': 2,
 'active_shards': 2,
 'relocating_shards': 0,
 'initializing_shards': 0,
 'unassigned_shards': 2,
 'delayed_unassigned_shards': 0,
 'number_of_pending_tasks': 0,
 'number_of_in_flight_fetch': 0,
 'task_max_waiting_in_queue_millis': 0,
 'active_shards_percent_as_number': 50.0}

## 2. Analysys

In [2]:
es.indices.close(index=index_name)

es.indices.put_settings(
    index=index_name,
    body={
        "analysis": {
            "filter": {},
            "analyzer": {
                "keyword_analyzer": {
                    "filter": [
                        "lowercase",
                        "asciifolding",
                        "trim"
                    ],
                    "char_filter": [],
                    "type": "custom",
                    "tokenizer": "keyword"
                },
                "edge_ngram_analyzer": {
                    "filter": [
                        "lowercase"
                    ],
                    "tokenizer": "edge_ngram_tokenizer"
                },
                "edge_ngram_search_analyzer": {
                    "tokenizer": "lowercase"
                }
            },
            "tokenizer": {
                "edge_ngram_tokenizer": {
                    "type": "edge_ngram",
                    "min_gram": 2,
                    "max_gram": 5,
                    "token_chars": [
                        "letter"
                    ]
                }
            }
        }
    })

es.indices.open(index=index_name)

/Users/cpu349/Desktop/PythonProject/venv/lib/python3.9/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: the default value for the ?wait_for_active_shards parameter will change from '0' to 'index-setting' in version 8; specify '?wait_for_active_shards=index-setting' to adopt the future default behaviour, or '?wait_for_active_shards=0' to preserve today's behaviour
  warnings.warn(message, category=ElasticsearchWarning)


{'acknowledged': True, 'shards_acknowledged': True}

## 2. Mapping (explicit)

In [3]:
es.indices.put_mapping(
    index=index_name,
    doc_type=type_name,
    include_type_name=True,
    body={
        type_name: {
            "properties": {
                "title": {
                    "type": "text",
                    "term_vector": "with_positions_offsets",
                    "fields": {
                        "keywordstring": {
                            "type": "text",
                            "analyzer": "keyword_analyzer"
                        },
                        "edgengram": {
                            "type": "text",
                            "analyzer": "edge_ngram_analyzer",
                            "search_analyzer": "edge_ngram_search_analyzer"
                        },
                        "completion": {
                            "type": "completion",
                            "contexts": [
                                {
                                    "name": "framework",
                                    "type": "category",
                                    "path": "tags"
                                },
                            ]
                        }
                    },
                    "analyzer": "standard"
                },
                "code": {
                    "type": "text",
                    "term_vector": "with_positions_offsets",
                },
                "date": {
                    "type": "date",
                    "format": "MMM dd yyyy"},
                "vote": {
                    "type": "integer",
                },
                "tags": {"type": "keyword"},
                "posted_by": {
                    "type": "text",
                    "term_vector": "with_positions_offsets",
                },
                "source_name": {
                    "type": "text",
                    "term_vector": "with_positions_offsets",
                },
                "source_url": {
                    "type": "text",
                    "term_vector": "with_positions_offsets",
                },
            }
        }
    }
)

/Users/cpu349/Desktop/PythonProject/venv/lib/python3.9/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: [types removal] Using include_type_name in put mapping requests is deprecated. The parameter will be removed in the next major version.
  warnings.warn(message, category=ElasticsearchWarning)


{'acknowledged': True}

## 5. Bulk insert scraped data

In [5]:
import json

from elasticsearch.helpers import bulk


def read_data(file):
    with open(file, 'r') as f:
        jobs = json.loads(f.read())
        res_list = [i for n, i in enumerate(jobs) if i not in jobs[n + 1:]]
        return res_list


def gen_data(job_list):
    for job in job_list:
        title = job.get("title", None)
        code = job.get("code", None)
        date = job.get("date", None)
        vote = job.get("vote", None)
        tags = job.get("tags", None)
        posted_by = job.get("posted_by", None)
        source_name = job.get("source_name", None)
        source_url = job.get("source_url", None)

        yield {
            "_index": index_name,
            "_source": {
                "title": title,
                "code": code,
                "date": date,
                "vote": vote,
                "tags": tags,
                "posted_by": posted_by,
                "source_name": source_name,
                "source_url": source_url,
            },
        }


import os
for root, dirs, files in os.walk('./data', topdown=False):
        for name in files:
            file = os.path.join(root, name)
            data = read_data(file)
            bulk(es, gen_data(data))

/Users/cpu349/Desktop/PythonProject/venv/lib/python3.9/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.13/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


ConnectionError: ConnectionError(<urllib3.connection.HTTPConnection object at 0x12de03160>: Failed to establish a new connection: [Errno 61] Connection refused) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x12de03160>: Failed to establish a new connection: [Errno 61] Connection refused)

# Test searching

In [ ]:
from pprint import pprint

results = es.search(
    index=index_name,
    doc_type=type_name,
    body={
        "query": {
            "match_all": {}
        }
    }
)
pprint(results)

In [ ]:
from pprint import pprint

results = es.search(
    index=index_name,
    # doc_type=type_name,
    body={
        "query": {
            "term": {
                "title": {
                    "value": "loop"
                }
            }
        },
        "script_score": {
            "script": {
                "source": "((doc['vote'].value - (-3))*380)/(380 - (-3))"
            }
        },
        "boost_mode": "sum",

        "aggs": {
            "min_vote": {
                "min": {
                    "field": "vote"
                }
            },
            "max_vote": {
                "max": {
                    "field": "vote"
                }
            },

            "query": {
                "bool": {
                    "must": [
                        {
                            "match": {
                                "title": "loop"
                            }
                        }
                    ],
                    "filter": [
                        {
                            "term": {
                                "tags": "angular"
                            }
                        }
                    ]
                }
            }
        }
    }
)

pprint(results)
